# 第12章 Excel集成

## 12.1 基本电子表格交互

### 12.1.1 生成工作簿（.xls）

In [6]:
import numpy as np
import pandas as pd
import xlrd, xlwt
import xlsxwriter

path = 'data/12/'

wb = xlwt.Workbook()
wb.add_sheet('first_sheet', cell_overwrite_ok=True)
wb.get_active_sheet()
ws_1 = wb.get_sheet(0)
ws_2 = wb.add_sheet('second_sheet', cell_overwrite_ok=True)

data = np.arange(1, 65).reshape((8, 8))

# 使用 write 方法提供行/列信息（使用零基索引）
ws_1.write(1, 0, 100)  # wirte 100 in cell "A1"
ws_2.write(0, 0, 100)

for c in range(data.shape[0]):
    for r in range(data.shape[1]):
        ws_1.write(r, c, int(data[c, r]))
        ws_2.write(r, c, int(data[r, c]))

wb.save(path + 'workbook.xls')

### 12.1.2 生成工作簿（.xslx）

In [7]:
# 创建一个工作簿
wb = xlsxwriter.Workbook(path + 'workbook.xlsx')
# 创建工作表对象
ws_1 = wb.add_worksheet('first_sheet')
ws_2 = wb.add_worksheet('second_sheet')
# 将数据写入到工作表对象
for c in range(data.shape[0]):
    for r in range(data.shape[1]):
        ws_1.write(r, c, data[c, r])
        ws_2.write(c, r, data[r, c])

wb.close()

#### 生成图表

In [8]:
wb = xlsxwriter.Workbook(path + 'chart.xlsx')
ws = wb.add_worksheet()

# write cumsum of random values in first column
values = np.random.standard_normal(15).cumsum()
ws.write_column('A1', values)

# create a new chart object
chart = wb.add_chart({'type': 'line'})

# add a series to the chart
chart.add_series({'values': '=Sheet1!$A$1:$A$15',
                  'marker': {'type': 'diamond'}, })

# insert the chart
ws.insert_chart('C1', chart)
wb.close()

### 12.1.3 从工作簿中读取

In [9]:
book = xlrd.open_workbook(path + 'workbook.xlsx')
book.sheet_names()
# ['first_sheet', 'second_sheet']
sheet_1 = book.sheet_by_name('first_sheet')
sheet_2 = book.sheet_by_index(1)

sheet_2.name
# 'second_sheet'

#  ncols 和 nrows 分别表示包含数据的列数和行数
sheet_1.ncols, sheet_1.nrows
# (8, 8)

cl = sheet_1.cell(0, 0)
cl.value
# 1.0

cl.ctype
# 2

2

#### Excel单元格类型

| 类型           | 编号 | Python类型                         |
| -------------- | ---- | ---------------------------------- |
| XL_CELL_EMPTY  | 0    | 空字符串                           |
| XL_CELL_TEXT   | 1    | Unicoe字符串                       |
| XL_CELL_NUMBER | 2    | float                              |
| XL_CELL_DATE   | 3    | foat                               |
| XL_CELL_BOLEAN | 4    | int(l=TRUE,0=FALSE)                |
| XL_CELL_ERROR  | 5    | int表示Excel内部编码               |
| XL_CELL_BLANK  | 6    | 空字符串，仅当formatting_info=True |

#### 可以为row方法提供行号，访问整行

In [10]:
sheet_2.row(3)

[number:4.0,
 number:12.0,
 number:20.0,
 number:28.0,
 number:36.0,
 number:44.0,
 number:52.0,
 number:60.0]

#### 访问整列

In [11]:
sheet_2.col(3)

[number:25.0,
 number:26.0,
 number:27.0,
 number:28.0,
 number:29.0,
 number:30.0,
 number:31.0,
 number:32.0]

#### row_values 和 col_values 只提供包含在对应行或者列的值

In [13]:
v1 = sheet_1.col_values(3, start_rowx=3, end_rowx=7)
v2 = sheet_1.row_values(3, start_colx=3, end_colx=7)

print(v1)
print(v2)

[28.0, 29.0, 30.0, 31.0]
[28.0, 36.0, 44.0, 52.0]


#### 读出工作簿对象中的所有数据

In [16]:
for c in range(sheet_1.ncols):
    for r in range(sheet_1.nrows):
        print('%i' % sheet_1.cell(r, c).value, end=', ')
    print()

1, 2, 3, 4, 5, 6, 7, 8, 
9, 10, 11, 12, 13, 14, 15, 16, 
17, 18, 19, 20, 21, 22, 23, 24, 
25, 26, 27, 28, 29, 30, 31, 32, 
33, 34, 35, 36, 37, 38, 39, 40, 
41, 42, 43, 44, 45, 46, 47, 48, 
49, 50, 51, 52, 53, 54, 55, 56, 
57, 58, 59, 60, 61, 62, 63, 64, 


### 12.1.4 使用OpenPyxl

In [17]:
import openpyxl as oxl

# 生成一个工作簿对象
wb = oxl.Workbook()
# 创建一个工作表对象
ws = wb.create_sheet(index=0, title='oxl_sheet')
# 将数据写入工作表
for c in range(data.shape[0]):
    for r in range(data.shape[1]):
        # 索引从1开始
        ws.cell(row=r+1, column=c+1).value = data[c, r]
        # create a Cell object and assigns a value

wb.save(path+'oxl_book.xlsx')

In [22]:
wb = oxl.load_workbook(path + 'oxl_book.xlsx')
ws = wb.active
cell = ws['B4']
print(cell.column)
# 'B'
print(cell.row)
# 4
print(cell.value)
# 12
print(ws['B1':'B4'])
for cell in ws['B1':'B4']:
    print(cell[0].value)

2
4
12
((<Cell 'oxl_sheet'.B1>,), (<Cell 'oxl_sheet'.B2>,), (<Cell 'oxl_sheet'.B3>,), (<Cell 'oxl_sheet'.B4>,))
9
10
11
12


### 12.1.5 使用pandas读写

In [27]:
df_1 = pd.read_excel(path + 'workbook.xlsx', 'first_sheet', header=None)
df_2 = pd.read_excel(path + 'workbook.xlsx', 'second_sheet', header=None)

import string

columns = []
data = np.arange(1, 65).reshape((8, 8))

for c in range(data.shape[0]):
    columns.append(string.ascii_uppercase[c])

print(columns)
# ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

df_1.columns = columns
df_2.columns = columns

df_1.to_excel(path + 'new_book_1.xlsx', 'my_sheet')

# 将多个DataFrame对象写人单个电子表格文件

wbw = pd.ExcelWriter(path + 'new_book_2.xlsx')
df_1.to_excel(wbw, 'first_sheet')
df_2.to_excel(wbw, 'second_sheet')

wbw.save()

# 检查一下， 是否已经在单一电子表格文件中生成了两个工作表：
wbn = xlrd.open_workbook(path + 'new_book_2.xlsx')
wbn.sheet_names()
# ['first_sheet', 'second_sheet']

# 更大量数据的读写
data = np.random.rand(20, 10000)
print(data.nbytes)
# 16000000

df = pd.DataFrame(data)
%time df.to_excel(path+'data.xlsx','data_sheet')
# Wall time: 58.6 s
%time np.save(path+'data',data)
# Wall time: 8 ms

%time df=pd.read_excel(path+'data.xlsx','data_sheet')
# Wall time: 6.53 s
%time data=np.load(path+'data.npy')
# Wall time: 10 ms

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
1600000
CPU times: user 3.67 s, sys: 161 ms, total: 3.83 s
Wall time: 3.89 s
CPU times: user 725 µs, sys: 2.18 ms, total: 2.91 ms
Wall time: 2.55 ms
CPU times: user 2.7 s, sys: 27 ms, total: 2.72 s
Wall time: 2.75 s
CPU times: user 636 µs, sys: 973 µs, total: 1.61 ms
Wall time: 1.47 ms
